# Web Scrapping para boletines de campaña de AMLO

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
# from tqdm import tqdm
from uuid import uuid4
import pandas as pd
import datetime as dt

## Definición de constantes

In [2]:
def obtener_ruta(fecha):
    return SITIO_BASE + fecha.strftime("%Y/%m/%d")

# FECHA_INICIO = dt.date(2018, 3, 30) # Inicio oficial
FECHA_INICIO = dt.date(2018, 4, 1)
FECHA_FIN = dt.date(2018, 6, 27)
UN_DIA = dt.timedelta(days=1)
DIAS_DIF = (FECHA_FIN - FECHA_INICIO).days + 1
SITIO_BASE = 'https://lopezobrador.org.mx/'
RUTA_CHROMEDRIVER = "C:\\Chromedriver\\chromedriver.exe"
RUTAS = [obtener_ruta(FECHA_INICIO + UN_DIA*i) for i in range(DIAS_DIF)]
articulos = []

## Driver chrome


In [3]:
driver = webdriver.Chrome(RUTA_CHROMEDRIVER)

## Scrapping
Lógica principal para obtener información

In [6]:
for ruta in tqdm(RUTAS, desc='Progreso por fecha'):
#     ruta_actual = obtener_ruta(fecha_actual)
    driver.get(ruta)
    content = driver.page_source
    soup = BeautifulSoup(content)
    
    # Buscamos todos los artículos con clase boletin
    for soup_articulo in soup.findAll('article', attrs={'class':'category-boletines'}):
        articulo = {
            'uuid': str(uuid4()),
            'fuente' : '',
            'fecha' : ruta[-10:], # En esta parte de la ruta se encuentra la fecha
            'encabezado' : '',
            'sumarios' : [],
            'contenido' : "",
        }
        
        # Encabezado del boletin
        soup_encabezado = soup_articulo.find('h2', attrs={'class':'entry-title'})
        articulo['encabezado'] = soup_encabezado.a.string
        articulo['fuente'] = soup_encabezado.a.attrs['href']
        
        # Sumarios del boletin
        soup_sumarios = soup_articulo.find('ul')
        if soup_sumarios:
            for soup_sumario in soup_sumarios.findAll('li'):
                sumario = soup_sumario.string if soup_sumario.string else ""
                if( len(sumario) < 3 ):
                    sumario = " ".join([child.string for child in soup_sumario.children if child.string])
                sumario = sumario.strip()
                if( len(sumario) >= 3):
                    articulo['sumarios'].append(sumario)
                
        # Contenido del boletin
        soup_content = soup_articulo.find('div', attrs={'class':'entry-content'})
        for soup_p in soup_content.findAll('p'):
            parrafo = soup_p.string if soup_p.string else ""
            if( len(parrafo) < 3 ):
                parrafo = " ".join([child.string for child in soup_p.children if child.string])
            parrafo = parrafo.strip()
            if( len(parrafo) >= 3 ): #and parrafo != 'ªªªªªªªªªªª' ):
                articulo['contenido'] += parrafo+"\n"
        
        # Agregamos el nuevo artículo
        articulos.append(articulo)
#     fecha_actual += UN_DIA
#     break
# Hacemos un objeto para boletines
boletines = {
    'boletines' : articulos
}

In [11]:
# Boletin faltante
boletines['boletines'][60]['contenido'] = """Boletín campaña-062
Descargar: Boletín campaña 062
Descargar: Entrevista AMLO, Coparmex  
Ciudad de México, 17 de mayo de 2018.-   El candidato por la coalición “Juntos haremos historia” por la Presidencia de México, Andrés Manuel López Obrador expresó que respeta la postura de Margarita Zavala, “muchos de los que iban a votar por ella, estoy seguro que lo van a hacer ahora por nosotros”.
En entrevista y a la pregunta de los reporteros sobre si es un acto de valentía o de congruencia Margarita Zavala a estas alturas, el candidato presidencial respondió que no lo califica, eso se los dejó a la interpretación de los representantes de los medios de comunicación.
Sobre si la ex candidata independiente pactó con Anaya, ayer prácticamente en todo el programa de Tercer Grado habló en contra de usted, López Obrador comentó que Zavala trae un cargo de conciencia.
Agregó: “porque su esposo Felipe Calderón se robó la Presidencia en el 2006, fue el que desató esta guerra que ha causado la muerte de miles de mexicanos, entonces Calderón convirtió al país en un cementerio, entonces, eso les pesa mucho, trae ese problema de conciencia, pero no es conmigo”.
Indicó que Margarita Zavala y Felipe Calderón deberían de entender que tienen esa deuda histórica con el pueblo de México.
A la pregunta de los reporteros sobre si cree que Zavala se vaya a sumar algún otro candidato, el candidato presidencial respondió que puede ser, porque se identifica con el PRIAN. “No sucede nada, ya la gente decidió a favor de un cambio verdadero, vamos a ganar aunque se unan todos, les vamos a ganar”.
Sobre si espera más retiros, más declinaciones de los candidatos, López Obrador manifestó que puede ser, porque están muy nerviosos, están muy preocupados, “estamos muy arriba, ya pasamos en términos de resultados finales de efectividad, ya pasamos el 50 por ciento de intención del voto”.
Expuso que está mucho muy arriba, desde luego es el ciudadano que decidirá, pero nunca se había visto en la historia del país que un candidato sacará tanta ventaja como ahora. “No es para presumir pobremente estamos como 30 puntos arriba”.
Comentó que fue una buena reunión con los integrantes de la Confederación Patronal de la República Mexicana (COPARMEX), fueron muy cordiales a pesar de que tiene diferencias, porque así es la democracia, se diálogo en un ambiente de respeto y de cordialidad.
ªªªªªªªªªªª"""

In [12]:
boletines['boletines'][60]

{'uuid': '8cb46087-5cd3-4a0c-9667-c30c2553fe40',
 'fuente': 'https://lopezobrador.org.mx/2018/05/17/expresa-amlo-que-respeta-la-postura-de-margarita-zavala/',
 'fecha': '2018/05/17',
 'encabezado': 'Expresa AMLO que respeta la postura de Margarita Zavala',
 'sumarios': ['Prevé que haya más declinaciones, están muy nerviosos “ya pasamos el 50 por ciento de intención del voto”',
  'Comentó que fue una buena reunión con los integrantes de la Confederación Patronal de la República Mexicana (COPARMEX)'],
 'contenido': 'Boletín campaña-062\nDescargar: Boletín campaña 062\nDescargar: Entrevista AMLO, Coparmex  \nCiudad de México, 17 de mayo de 2018.-   El candidato por la coalición “Juntos haremos historia” por la Presidencia de México, Andrés Manuel López Obrador expresó que respeta la postura de Margarita Zavala, “muchos de los que iban a votar por ella, estoy seguro que lo van a hacer ahora por nosotros”.\nEn entrevista y a la pregunta de los reporteros sobre si es un acto de valentía o de

## Guardar resultados

In [7]:
df_boletines = pd.DataFrame(boletines)

In [8]:
with open('Data/boletines-AMLO.json', 'w', encoding='utf-8') as file:
    df_boletines.to_json(file, force_ascii=False)